# Predictive Maintenance System

## Objetivo

Este projeto visa implementar um sistema de manutenção preditiva utilizando dados de sensores. O código está estruturado para ser executado diretamente em um ambiente Jupyter Notebook. Ao executar cada célula, o código realiza todas as etapas necessárias para a configuração, geração de dados, treinamento de modelos e geração de relatórios.

## Estrutura do Projeto

1. **Configuração de Caminho:** Define o caminho para armazenar dados simulados e garante que o diretório necessário esteja criado.
2. **Geração de Dados Simulados:** Cria e adiciona novos dados simulados a um arquivo CSV.
3. **Carregamento e Análise Exploratória de Dados:** Carrega os dados existentes e realiza uma análise inicial, visualizando estatísticas e gráficos.
4. **Processamento de Dados:** Limpa e normaliza os dados, e realiza engenharia de características.
5. **Desenvolvimento do Modelo:**
   - **Divisão dos Dados:** Separa os dados em conjuntos de treinamento e teste.
   - **Treinamento do Modelo:** Treina um modelo de RandomForestClassifier com os dados de treinamento.
   - **Avaliação do Modelo:** Avalia o desempenho do modelo com o conjunto de teste.
6. **Implementação com TensorFlow:** Constrói e treina uma rede neural usando TensorFlow.
7. **Geração de Gráficos de Desempenho do Modelo:** Cria gráficos para visualização da precisão e perda durante o treinamento.
8. **Geração de Relatório em PDF:** Compila um relatório em PDF contendo resultados do modelo e gráficos gerados.
9. **Coleta Simulada de Dados e Envio de Alertas:** Simula a coleta contínua de dados dos sensores e envia alertas baseados nas previsões do modelo.

## Como Executar

Para executar o notebook:

1. **Inicie o notebook Jupyter.**
2. **Execute cada célula sequencialmente (pressionando `Shift + Enter`).** 
   - As células irão configurar o ambiente, gerar e processar dados, treinar e avaliar modelos, e finalmente gerar gráficos e relatórios.

Todas as etapas são autossuficientes e o código está preparado para ser executado diretamente. Certifique-se de que todas as bibliotecas necessárias estão instaladas e o ambiente está configurado corretamente.

## Requisitos

Certifique-se de que as seguintes bibliotecas estão instaladas:

- `numpy`
- `pandas`
- `scikit-learn`
- `tensorflow`
- `requests`
- `fpdf`
- `matplotlib`

Você pode instalar as bibliotecas necessárias com o comando:
```bash
pip install numpy pandas scikit-learn tensorflow requests fpdf matplotlib


## 1. Configuração de caminho

In [ ]:
import os

# Definindo o caminho relativo ao diretório de dados
base_dir = os.path.join(os.getcwd(), '..', 'data')

# Verificando se o diretório existe; se não, cria
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Definindo o caminho completo para o arquivo CSV
file_path = os.path.join(base_dir, 'sensor_data.csv')


## 2. Gerando dados simulados

In [ ]:
import numpy as np
import pandas as pd

# Definindo parâmetros para os novos dados
np.random.seed(45)
num_samples = 1000

# Função para carregar dados existentes e adicionar novos dados
def append_to_csv(file_path, new_data):
    if os.path.exists(file_path):
        existing_df = pd.read_csv(file_path)
        new_df = pd.DataFrame(new_data)
        updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    else:
        updated_df = pd.DataFrame(new_data)

    updated_df.to_csv(file_path, index=False)
    print(f"Dados atualizados salvos em: {file_path}")

# Gerar novos dados simulados
temperature = np.random.normal(loc=70, scale=5, size=num_samples)
vibration = np.random.normal(loc=30, scale=2, size=num_samples)
pressure = np.random.normal(loc=100, scale=10, size=num_samples)
failure = np.random.choice([0, 1], size=num_samples, p=[0.95, 0.05])

# Criar o DataFrame com os novos dados
new_data = {
    'temperature': temperature,
    'vibration': vibration,
    'pressure': pressure,
    'failure': failure
}

# Adicionar os novos dados ao CSV existente
append_to_csv(file_path, new_data)


## 3. Carregamento e Análise Exploratória de Dados


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(df.head())
    print(df.describe())
    df.plot(subplots=True, figsize=(10, 12))
    plt.show()
else:
    print(f"Arquivo não encontrado no caminho: {file_path}")


## 4. Processamento de Dados

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Removendo valores ausentes
df = df.dropna()

# Normalizando os dados
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Feature Engineering
df_scaled['temperature_vibration_ratio'] = df_scaled['temperature'] / df_scaled['vibration']


## 5. Desenvolvimento do Modelo


### Divisão dos Dados:



In [ ]:
from sklearn.model_selection import train_test_split

X = df_scaled.drop('failure', axis=1)  # Features
y = df_scaled['failure']  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### Verificação e o tratamento de dados após a divisão:

In [ ]:
import numpy as np

X = df[['temperature', 'vibration', 'pressure']]
y = df['failure']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.isnull().sum())
print(np.isinf(X_train).sum())

X_train = X_train.fillna(X_train.mean())
X_train = X_train.replace([np.inf, -np.inf], np.nan).dropna()

print(y_train.isnull().sum())
print(np.isinf(y_train).sum())

y_train = y_train.fillna(y_train.mean())
y_train = y_train.replace([np.inf, -np.inf], np.nan).dropna()


### Treinamento do Modelo:



In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


### Avaliação do Modelo:



In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


## 6. Implementação com TensorFlow


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')


## 7. Geração de Gráficos de Desempenho do modelo


In [ ]:
import matplotlib.pyplot as plt
import os

def criar_graficos(history):
    plt.figure(figsize=(10, 4))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig(os.path.join(base_dir, 'accuracy_plot.png'))
    plt.close()

    plt.figure(figsize=(10, 4))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(base_dir, 'loss_plot.png'))
    plt.close()

criar_graficos(history)


## 8. Gerando o Relatório em PDF do modelo


In [ ]:
from fpdf import FPDF
import os

def gerar_relatorio(accuracy, loss):
    pdf = FPDF()
    pdf.add_page()

    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Relatório de Resultados do Modelo", ln=True, align='C')

    pdf.ln(10)
    pdf.cell(200, 10, txt=f"Precisão no Teste: {accuracy*100:.2f}%", ln=True)
    pdf.cell(200, 10, txt=f"Perda no Teste: {loss:.4f}", ln=True)

    pdf.ln(10)
    pdf.cell(200, 10, txt="Gráficos de Treinamento", ln=True)
    
    pdf.image(os.path.join(base_dir, 'accuracy_plot.png'), x=10, y=60, w=180)
    pdf.add_page()
    pdf.image(os.path.join(base_dir, 'loss_plot.png'), x=10, y=10, w=180)

    pdf.output(os.path.join(base_dir, "relatorio_resultados.pdf"))
    print("Relatório PDF gerado com sucesso.")

gerar_relatorio(accuracy, loss)


## 9. Coleta Simulada de Dados e Envio de Alertas

In [ ]:
import numpy as np
import pandas as pd
import requests
import time

def initialize_csv(file_path):
    if not os.path.exists(file_path):
        df = pd.DataFrame(columns=['temperature', 'vibration', 'pressure', 'failure'])
        df.to_csv(file_path, index=False)

initialize_csv(file_path)

def send_alert(message):
    url = "https://api.smsgateway/send"  # URL da API do serviço de alerta
    data = {
        "message": message,
        "to": "numero_do_telefone"
    }
    response = requests.post(url, data=data)
    if response.status_code == 200:
        print("Alerta enviado com sucesso!")
    else:
        print("Falha ao enviar alerta.")

def process_new_data(sensor_data):
    processed_data = np.array(sensor_data[:-1]).reshape(1, -1)
    prediction = model.predict(processed_data)
    print(f"Actual prediction: {prediction}")

    if prediction > 0.5:
        print("Alerta: Possível falha detectada. Verifique a máquina.")
        # send_alert("Alerta: Possível falha detectada. Verifique a máquina.")
    else:
        print("Nenhuma falha detectada.")

    return sensor_data

def simulate_sensor_data():
    while True:
        temperature = np.random.normal(loc=70, scale=5)
        vibration = np.random.normal(loc=30, scale=2)
        pressure = np.random.normal(loc=100, scale=10)
        failure = np.random.choice([0, 1], size=1, p=[0.95, 0.05])[0]

        sensor_data = [temperature, vibration, pressure, failure]
        processed_data = process_new_data(sensor_data)
        df = pd.DataFrame([processed_data], columns=['temperature', 'vibration', 'pressure', 'failure'])
        append_to_csv(file_path, df.to_dict(orient='records'))
        
        time.sleep(10)  # Intervalo de coleta de dados (10 segundos)

simulate_sensor_data()
